# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,chokurdakh,70.6333,147.9167,-14.03,96,100,3.33,RU,1700009420
1,1,saint-philippe,-21.3585,55.7679,22.77,84,41,4.01,RE,1700009420
2,2,busselton,-33.6500,115.3333,20.60,56,42,7.44,AU,1700009421
3,3,qaanaaq,77.4840,-69.3632,-20.78,53,11,3.98,GL,1700009422
4,4,tuktoyaktuk,69.4541,-133.0374,-11.00,92,20,3.09,CA,1700009423


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
mi_temp_s = city_data_df["Max Temp"] > 21
ma_temp_s = city_data_df["Max Temp"] < 27
w_speed_s = city_data_df["Wind Speed"] > 5

print(sum(mi_temp_s))
print(sum(ma_temp_s))
print(sum(w_speed_s))

225
499
140


In [11]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp_range_df = city_data_df[mi_temp_s & ma_temp_s & w_speed_s]
city_temp_range_df

# Drop any rows with null values
clean_city_temp_range_df = city_temp_range_df.dropna()

# Display sample data
clean_city_temp_range_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,faanui,-16.4833,-151.7500,26.80,80,100,7.18,PF,1700009426
30,30,vaini,-21.2000,-175.2000,25.09,100,100,7.72,TO,1700009442
37,37,carmen,18.6333,-91.8333,25.90,86,7,5.94,MX,1700009378
41,41,sao joao da barra,-21.6403,-41.0511,26.05,84,8,9.42,BR,1700009449
48,48,touros,-5.1989,-35.4608,26.56,81,100,6.22,BR,1700009453
54,54,rikitea,-23.1203,-134.9692,24.33,86,100,8.36,PF,1700009457
95,95,port-gentil,-0.7193,8.7815,26.81,77,100,5.38,GA,1700009485
108,108,hilo,19.7297,-155.0900,26.27,85,100,6.69,US,1700009211
135,135,fare,-16.7000,-151.0167,26.76,80,100,6.73,PF,1700009513
152,152,atuona,-9.8000,-139.0333,26.71,73,7,5.15,PF,1700009527


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,faanui,-16.4833,-151.7500,26.80,80,100,7.18,PF,1700009426,
30,30,vaini,-21.2000,-175.2000,25.09,100,100,7.72,TO,1700009442,
37,37,carmen,18.6333,-91.8333,25.90,86,7,5.94,MX,1700009378,
41,41,sao joao da barra,-21.6403,-41.0511,26.05,84,8,9.42,BR,1700009449,
48,48,touros,-5.1989,-35.4608,26.56,81,100,6.22,BR,1700009453,
54,54,rikitea,-23.1203,-134.9692,24.33,86,100,8.36,PF,1700009457,
95,95,port-gentil,-0.7193,8.7815,26.81,77,100,5.38,GA,1700009485,
108,108,hilo,19.7297,-155.0900,26.27,85,100,6.69,US,1700009211,
135,135,fare,-16.7000,-151.0167,26.76,80,100,6.73,PF,1700009513,
152,152,atuona,-9.8000,-139.0333,26.71,73,7,5.15,PF,1700009527,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = "accomodation.hotel"
limit = 1
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accomodation.hotel&limit=1&apiKey=8236527520ba4a53ad3610db3c304229&filter=circle%3A-151.75%2C-16.4833%2C10000&bias=proximity%3A-151.75%2C-16.4833
faanui - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accomodation.hotel&limit=1&apiKey=8236527520ba4a53ad3610db3c304229&filter=circle%3A-175.2%2C-21.2%2C10000&bias=proximity%3A-175.2%2C-21.2
vaini - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accomodation.hotel&limit=1&apiKey=8236527520ba4a53ad3610db3c304229&filter=circle%3A-91.8333%2C18.6333%2C10000&bias=proximity%3A-91.8333%2C18.6333
carmen - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=accomodation.hotel&limit=1&apiKey=8236527520ba4a53ad3610db3c304229&filter=circle%3A-41.0511%2C-21.6403%2C10000&bias=proximity%3A-41.0511%2C-21.6403
sao joao da barra - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?categories=acco

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,10,faanui,-16.4833,-151.7500,26.80,80,100,7.18,PF,1700009426,No hotel found
30,30,vaini,-21.2000,-175.2000,25.09,100,100,7.72,TO,1700009442,No hotel found
37,37,carmen,18.6333,-91.8333,25.90,86,7,5.94,MX,1700009378,No hotel found
41,41,sao joao da barra,-21.6403,-41.0511,26.05,84,8,9.42,BR,1700009449,No hotel found
48,48,touros,-5.1989,-35.4608,26.56,81,100,6.22,BR,1700009453,No hotel found
54,54,rikitea,-23.1203,-134.9692,24.33,86,100,8.36,PF,1700009457,No hotel found
95,95,port-gentil,-0.7193,8.7815,26.81,77,100,5.38,GA,1700009485,No hotel found
108,108,hilo,19.7297,-155.0900,26.27,85,100,6.69,US,1700009211,No hotel found
135,135,fare,-16.7000,-151.0167,26.76,80,100,6.73,PF,1700009513,No hotel found
152,152,atuona,-9.8000,-139.0333,26.71,73,7,5.15,PF,1700009527,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600, 
    scale = 0.5,
    color = "City"
)

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)